# Delta Practice

In [2]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
tvShowDF = pd.read_excel(filepath1)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

In [3]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [4]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=90)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

tvShowDF = DateTimeConvert(tvShowDF,'RunDate')

In [5]:
tvShowDF.sort_values(['TvShow', 'RunDate'], inplace=True)

tvShowDF['diffs'] = tvShowDF.groupby(['TvShow'])['HashtagValue'].transform(lambda x: x.diff()) .fillna(0)

tvShowDF.sort_index(inplace=True)

In [6]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [7]:
def MergeDfs(mainDf, secondDf):

    merged = pd.merge(mainDf,secondDf, how='outer', on=['TvShow'])
    return merged

In [8]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf[columnName] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [9]:
def SumSocialMedia(sumDf,columnName):

    summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
    summarizedDf = pd.DataFrame(summarizedDf)

    return summarizedDf

In [10]:
instaDf = RemoveData(releaseDf, tvShowDF)
instaDf = NegativeDiffs(instaDf, 'diffs')

iHashtagDf = SumSocialMedia(instaDf, 'TvShow')

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_73864/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


In [11]:
uberDf = MergeDfs(releaseDf, iHashtagDf)

In [12]:
uberDf.head(10)

,TvShow,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,90DayDate,HashtagValue,diffs
0,ABlackLadySketchShow,HBOMax,NaN,NaN,NaN,2022-04-08,2022-07-07,408351.0,464.0
1,AboutLastNighthbo,HBOMax,NaN,1.0,NaN,2022-02-10,2022-05-11,334.0,1.0
2,AdventureOfTheRing,HBOMax,NaN,1.0,NaN,2022-02-10,2022-05-11,14492.0,2.0
3,AllTheOldKnives,Prime,NaN,NaN,NaN,2022-04-08,2022-07-07,268196.0,2020.0
4,Archive81,Netflix,8.0,1.0,Binge,2022-01-14,2022-04-14,310066.0,982.0
5,Atlantafx,Hulu,NaN,3.0,NaN,2022-03-25,2022-06-23,2823307.0,1885.0
6,Barry,HBOMax,NaN,NaN,NaN,2022-04-24,2022-07-23,1175768.0,1327.0
7,BelAirPeacock,Peacock,10.0,1.0,Hybrid,2022-02-13,2022-05-14,250414.0,2894.0
8,BlingEmpire,Netflix,NaN,NaN,NaN,2022-05-13,2022-08-11,846954.0,2187.0
9,Bridgerton,Netflix,NaN,2.0,NaN,2022-03-25,2022-06-23,46065959.0,171916.0


filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf.to_csv(filepath2) 